In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [15]:
# Checkpoint file names
PRODUCT_URLS_FILE = 'shoe_urls_checkpoint.csv'
REVIEWS_FILE = 'shoe_reviews_checkpoint.csv'

In [16]:
# Function to create a session with retry logic
def create_session():
    session = requests.Session()
    retry = Retry(
        total=5,
        read=5,
        connect=5,
        backoff_factor=0.3,
        status_forcelist=(500, 502, 503, 504),
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

In [17]:
# Function to get product URLs from a single listing page
def get_product_urls(listing_url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(listing_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    product_urls = []
    
    for a_tag in soup.find_all('a', class_='core'):
        url = a_tag.get('href')
        if url:
            product_urls.append('https://www.jumia.com.ng' + url)
            
    return product_urls

In [18]:
# Function to get the maximum number of review pages for a product
def get_max_review_pages(product_url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(product_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return 1

    soup = BeautifulSoup(response.content, 'html.parser')
    pagination = soup.find('div', class_='pg-w -ptm -pbxl')
    if pagination:
        pages = pagination.find_all('a')
        page_numbers = [int(page.text.strip()) for page in pages if page.text.strip().isdigit()]
        if page_numbers:
            return max(page_numbers)
    return 1  # Default to 1 if pagination is not found


In [19]:
# Function to get product reviews from a single page
def get_product_reviews(url):
    session = create_session()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    
    reviews = []
    review_elements = soup.find_all('article', class_='-pvs -hr _bet')

    for element in review_elements:
        review = {}
        
        rating_tag = element.find('div', class_='stars _m _al -mvs')
        review['rating'] = rating_tag.text.split(' out of ')[0].strip() if rating_tag else 'N/A'
        
        title_tag = element.find('h3', class_='-m -fs16 -pvs')
        review['title'] = title_tag.text.strip() if title_tag else 'N/A'
        
        body_tag = element.find('p', class_='-pvs')
        review['body'] = body_tag.text.strip() if body_tag else 'N/A'
        
        date_tag = element.find('span', class_='-prs')
        review['date'] = date_tag.text.strip() if date_tag else 'N/A'
        
        name_tag = element.find('span', string=lambda x: x and x.startswith('by '))
        review['name'] = name_tag.text.replace('by ', '').strip() if name_tag else 'N/A'
        
        reviews.append(review)

    return reviews

In [20]:
# Function to get all reviews for a given product URL
def get_all_reviews(product_url):
    max_pages = get_max_review_pages(product_url)
    all_reviews = []
    for page in range(1, max_pages + 1):
        url = f"{product_url}?page={page}"
        reviews = get_product_reviews(url)
        all_reviews.extend(reviews)
        time.sleep(0.5)  # Add a delay to avoid hitting the server too quickly
    
    return all_reviews

In [21]:
# Function to save product URLs to a CSV file (checkpoint)
def save_product_urls_checkpoint(product_urls):
    df = pd.DataFrame(product_urls, columns=['product_url'])
    df.to_csv(PRODUCT_URLS_FILE, index=False)

In [22]:
# Function to load product URLs from a CSV file (checkpoint)
def load_product_urls_checkpoint():
    if os.path.exists(PRODUCT_URLS_FILE):
        df = pd.read_csv(PRODUCT_URLS_FILE)
        return df['product_url'].tolist()
    return []

In [23]:
# Function to save reviews to a CSV file (checkpoint)
def save_reviews_checkpoint(reviews):
    df = pd.DataFrame(reviews)
    if os.path.exists(REVIEWS_FILE):
        df.to_csv(REVIEWS_FILE, mode='a', header=False, index=False)
    else:
        df.to_csv(REVIEWS_FILE, index=False)

In [24]:
# Function to load reviews from a CSV file (checkpoint)
def load_reviews_checkpoint():
    if os.path.exists(REVIEWS_FILE):
        return pd.read_csv(REVIEWS_FILE).to_dict('records')
    return []

In [25]:
# Function to scrape multiple products
def scrape_multiple_products(product_urls):
    all_product_reviews = load_reviews_checkpoint()
    
    # Get already scraped product URLs to avoid reprocessing
    scraped_product_urls = {review['product_url'] for review in all_product_reviews}
    
    for product_url in product_urls:
        if product_url not in scraped_product_urls:
            print(f"Scraping reviews for: {product_url}")
            product_reviews = get_all_reviews(product_url)
            for review in product_reviews:
                review['product_url'] = product_url  # Add product URL to each review for checkpointing
            all_product_reviews.extend(product_reviews)
            save_reviews_checkpoint(product_reviews)  # Save progress
            scraped_product_urls.add(product_url)
    
    return all_product_reviews

In [26]:
def main():
    base_listing_url = 'https://www.jumia.com.ng/catalog/?q=shoe&page='  # Base URL for the listing pages
    total_listing_pages = 50  # Total number of listing pages to scrape

    # Load previously scraped product URLs to avoid reprocessing
    all_product_urls = load_product_urls_checkpoint()
    last_page_scraped = len(all_product_urls) // 40  # Assuming 40 products per page

    # Step 1: Get product URLs from all listing pages
    for page in range(last_page_scraped + 1, total_listing_pages + 1):
        listing_url = f"{base_listing_url}{page}"
        product_urls = get_product_urls(listing_url)
        all_product_urls.extend(product_urls)
        print(f"Page {page}: Found {len(product_urls)} product URLs.")
        save_product_urls_checkpoint(all_product_urls)  # Save progress
    
    print(f"Total product URLs found: {len(all_product_urls)}")

    # Step 2: Scrape reviews for each product URL
    all_reviews = scrape_multiple_products(all_product_urls)

    if all_reviews:
        df = pd.DataFrame(all_reviews)
        # Reorder the columns
        df = df[['name', 'rating', 'date', 'title', 'body', 'product_url']]
        print(df)
        # Save to a CSV file
        filename = os.path.join(os.getcwd(), 'jumia_NLP_shoesdataset.csv')
        df.to_csv(filename, index=False)
        print(f"File saved as {filename}")

        # Additional confirmation
        if os.path.exists(filename):
            print(f"Confirmed: {filename} has been saved.")
        else:
            print(f"Error: {filename} was not saved.")
    else:
        print("No reviews found.")

if __name__ == '__main__':
    main()

Page 1: Found 40 product URLs.
Page 2: Found 40 product URLs.
Page 3: Found 40 product URLs.
Page 4: Found 40 product URLs.
Page 5: Found 40 product URLs.
Page 6: Found 40 product URLs.
Page 7: Found 40 product URLs.
Page 8: Found 40 product URLs.
Page 9: Found 40 product URLs.
Page 10: Found 40 product URLs.
Page 11: Found 40 product URLs.
Page 12: Found 40 product URLs.
Page 13: Found 40 product URLs.
Page 14: Found 40 product URLs.
Page 15: Found 40 product URLs.
Page 16: Found 40 product URLs.
Page 17: Found 40 product URLs.
Page 18: Found 40 product URLs.
Page 19: Found 40 product URLs.
Page 20: Found 40 product URLs.
Page 21: Found 40 product URLs.
Page 22: Found 40 product URLs.
Page 23: Found 40 product URLs.
Page 24: Found 40 product URLs.
Page 25: Found 40 product URLs.
Page 26: Found 40 product URLs.
Page 27: Found 40 product URLs.
Page 28: Found 40 product URLs.
Page 29: Found 40 product URLs.
Page 30: Found 40 product URLs.
Page 31: Found 40 product URLs.
Page 32: Found 40

Scraping reviews for: https://www.jumia.com.ng/confirm-crack-chain-shoes-black-vangelo-mpg2039407.html
Scraping reviews for: https://www.jumia.com.ng/fashion-luxury-stoned-corporate-and-wedding-mens-shoe-black-335128379.html
Scraping reviews for: https://www.jumia.com.ng/generic-flangesio-fashion-tassel-loafers-men-dress-leather-shoes-italian-designer-hair-stylist-party-boats-shoes-lazy-shoes-men-driving-shoes-brown-355462669.html
Scraping reviews for: https://www.jumia.com.ng/generic-mens-pu-leather-free-slip-on-loafer-waterproof-formal-shoes-black-272410121.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-italian-plus-size-smooth-leather-shoes-brown-125189746.html
Scraping reviews for: https://www.jumia.com.ng/fashion-perfect-fit-casual-comfortable-fashionable-shoe-266702790.html
Scraping reviews for: https://www.jumia.com.ng/generic-adjustable-spring-shoe-stretchershoe-width-and-length-expander-267757207.html
Scraping reviews for: https://www.jumia.com.ng/fashion-gra

Scraping reviews for: https://www.jumia.com.ng/fashion-a-pair-of-adjustable-men-shoe-tree-and-expander-110019578.html
Scraping reviews for: https://www.jumia.com.ng/fashion-cristal-shimmer-girls-shoe-126671168.html
Scraping reviews for: https://www.jumia.com.ng/adidas-sports-performance-breaknet-teens-96894138.html
Scraping reviews for: https://www.jumia.com.ng/fashion-kids-shoes-boys-shoes-girls-shoes-children-shoes-160943137.html
Scraping reviews for: https://www.jumia.com.ng/fuce-unisex-casual-school-sandal-black-165299086.html
Scraping reviews for: https://www.jumia.com.ng/generic-men-soccer-shoes-ag-high-ankle-football-boots-men-sneakers-white-256664515.html
Scraping reviews for: https://www.jumia.com.ng/fashion-rising-star-school-shoe-unisex-boy-girl-kids-children-canvas-sneakers-black-117088862.html
Scraping reviews for: https://www.jumia.com.ng/fashion-highly-dignifying-male-child-shoe-95660275.html
Scraping reviews for: https://www.jumia.com.ng/fashion-unisex-children-sneakers

Scraping reviews for: https://www.jumia.com.ng/fashion-safety-boots-steel-toe-work-shoes-fabric-upper-ultralight-195189335.html
Scraping reviews for: https://www.jumia.com.ng/non-marking-shoe-anti-slip-padsole-guard-109646479.html
Scraping reviews for: https://www.jumia.com.ng/fashion-velcro-leather-sandal-brown-71799480.html
Scraping reviews for: https://www.jumia.com.ng/generic-men-soccer-shoes-high-ankle-football-boots-men-sneakers-201731036.html
Scraping reviews for: https://www.jumia.com.ng/santiago-mens-italian-leather-luxury-office-wedding-shoes-black-43652546.html
Scraping reviews for: https://www.jumia.com.ng/fashion-children-new-leather-school-shoe-black-19342953.html
Scraping reviews for: https://www.jumia.com.ng/adidas-grand-court-lifestyle-hook-and-loop-shoes-unisex-271976936.html
Scraping reviews for: https://www.jumia.com.ng/mens-men-shoes-fashion-trendy-skate-sneakers-shoes-red-216619895.html
Scraping reviews for: https://www.jumia.com.ng/fashion-unisex-childrens-canvas

Scraping reviews for: https://www.jumia.com.ng/adidas-performance-football-shoes-kids-x-18.3-in-j-92831251.html
Scraping reviews for: https://www.jumia.com.ng/adidas-core-neo-slippers-kids-fortaswim-2-c-92831386.html
Scraping reviews for: https://www.jumia.com.ng/mens-patent-leather-suede-dress-shoe-wedding-casual-office-shoe-black-varrati-mpg2039416.html
Scraping reviews for: https://www.jumia.com.ng/generic-2-way-unisex-adjustable-shoe-expander-with-accesseries-147479147.html
Scraping reviews for: https://www.jumia.com.ng/generic-durable-protective-shoe-covers-100pcs-235475679.html
Scraping reviews for: https://www.jumia.com.ng/mens-sports-canvas-casual-sneakers-white-171889463.html
Scraping reviews for: https://www.jumia.com.ng/ladies-everything-womens-fashion-walking-outdoor-sport-sneakers-casual-canvas-women-shoe-241891980.html
Scraping reviews for: https://www.jumia.com.ng/fashion-childrens-high-top-martin-short-boots-pink-217743656.html
Scraping reviews for: https://www.jumia.co

...

Scraping reviews for: https://www.jumia.com.ng/generic-men-cleats-soccer-shoes-team-sport-football-boots-training-fg-tf-turf-sneakers-270549625.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-retro-fashion-hiking-shoes-casual-outdoor-shoes-beige-121234529.html
Scraping reviews for: https://www.jumia.com.ng/looking-smart-ladies-flat-school-women-shoe-black-60366060.html
Scraping reviews for: https://www.jumia.com.ng/fashion-soft-soled-non-slip-leather-shoes-xf528-black-162287118.html
Scraping reviews for: https://www.jumia.com.ng/fashion-ladies-platform-plus-size-slippers-blue-350855131.html
Scraping reviews for: https://www.jumia.com.ng/fashion-new-teen-boys-soccer-shoes-men-football-sneakers-boots-red-240270868.html
Scraping reviews for: https://www.jumia.com.ng/fashion-childrens-comfortable-sandals-outdoor-shoes-200885704.html
Scraping reviews for: https://www.jumia.com.ng/generic-cloth-hanger-with-shoe-rack-190315518.html
Scraping reviews for: https://www.jumia.com.

Scraping reviews for: https://www.jumia.com.ng/vangelo-new-arrival-men-classic-shoe-black-202141934.html
Scraping reviews for: https://www.jumia.com.ng/mens-formal-shoes-businesss-office-dress-shoes-for-man-black-fashion-mpg3361001.html
Scraping reviews for: https://www.jumia.com.ng/fashion-9917-blackoxford-leather-shoes-men-breathable-patent-leather-formal-shoes-plus-size-man-office-wedding-flats-male-black-luxury-business-jin-256141135.html
Scraping reviews for: https://www.jumia.com.ng/new-shining-barge-designers-shoe-black-depally-mpg3355885.html
Scraping reviews for: https://www.jumia.com.ng/fashion-big-boys-casual-shoes-mesh-spiderman-childrens-shoes-red-77037664.html
Scraping reviews for: https://www.jumia.com.ng/fashion-whitemen-british-formal-dress-shoes-for-male-coiffeur-white-formal-loafers-classic-wedding-party-footwear-slip-on-plus-size-39-48-acu-255926260.html
Scraping reviews for: https://www.jumia.com.ng/stainless-shoe-rack-fabric-cover-6-layers-generic-mpg3286759.html


Scraping reviews for: https://www.jumia.com.ng/canvas-women-shoe-casual-canvas-mens-fashionable-sneakers-241893197.html
Scraping reviews for: https://www.jumia.com.ng/fashion-winter-children-warm-shoes-kids-male-waterproof-martin-boots-cotton-boots-108649425.html
Scraping reviews for: https://www.jumia.com.ng/vangelo-three-belt-quality-vangelo-designer-mens-shoe-black-337184850.html
Scraping reviews for: https://www.jumia.com.ng/fashion-boys-school-shoe-white-258657201.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-all-match-slip-on-canvas-shoes-201938919.html
Scraping reviews for: https://www.jumia.com.ng/fashion-classic-italian-chain-shoe-black-126448997.html
Scraping reviews for: https://www.jumia.com.ng/fashion-d-classik-barge-designers-shoes-black-259272148.html
Scraping reviews for: https://www.jumia.com.ng/fashion-high-quality-white-sneakers-with-free-sock-256589558.html
Scraping reviews for: https://www.jumia.com.ng/generic-stainless-shoe-rack-with-fabric-cove

Scraping reviews for: https://www.jumia.com.ng/generic-rack-cloth-hanger-and-storage-282117776.html
Scraping reviews for: https://www.jumia.com.ng/fashion-italian-mens-shoes-casual-luxury-brand-winter-warm-men-loafers-genuine-leather-moccasins-breathable-slip-on-boat-shoes-350904957.html
Scraping reviews for: https://www.jumia.com.ng/italian-silver-formal-shoes-for-men-eur-size-38-46-new-arrivals-gentleman-carved-bullock-business-dress-shoes-mens-wingtip-oxford-leather-shoes-elegant-wedding-shoes-adult-designer-male-social-footwear-flangesio-mpg4089655.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-leather-half-shoe-black-156405950.html
Scraping reviews for: https://www.jumia.com.ng/flangesio-eur-size-38-46-business-formal-shoes-for-men-handmade-calf-leather-oxfords-shoes-luxury-italian-style-male-dress-shoes-lace-up-soft-office-banquet-flats-shoes-groom-wedding-shoes-brown-262069432.html
Scraping reviews for: https://www.jumia.com.ng/viny-childrens-sandals-casual-134

Scraping reviews for: https://www.jumia.com.ng/clarks-smart-black-work-shoe-235275928.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-italian-plus-size-smooth-leather-shoes-brown-263739102.html
Scraping reviews for: https://www.jumia.com.ng/cubana-cool-bigger-boys-bukkle-shoe-black-fashion-mpg3361385.html
Scraping reviews for: https://www.jumia.com.ng/fashion-trend-patchwork-leather-dress-shoes-for-men-casual-business-office-formal-shoes-italian-designer-shoes-social-shoes-wedding-shoes-party-flats-brown-eur-size-38-48-flangesio-mpg4089641.html
Scraping reviews for: https://www.jumia.com.ng/new-cool-stone-tie-designers-shoe-black-depally-mpg3994626.html
Scraping reviews for: https://www.jumia.com.ng/fashion-mens-large-size-pointed-toe-carved-leather-brown-shoes-257802826.html
Scraping reviews for: https://www.jumia.com.ng/fashion-rain-boots-women-shallow-low-top-flat-bottom-hollow-sandals-shoes-black-242213415.html
Scraping reviews for: https://www.jumia.com.ng/fashion